<h1> Hyperparameter tuning </h1>

This notebook is Lab4b of CPB 102, Google's course on Machine Learning using Cloud ML.

This notebook builds on Lab 4a, adding hyperparameter tuning to the feature engineering done in that lab.  To save time, we will start from the preprocessed output of Lab 4a.

In [10]:
import google.cloud.ml as ml
import tensorflow as tf
print tf.__version__
print ml.sdk_location

0.11.0rc0
gs://cloud-ml/sdk/cloudml-0.1.6-alpha.dataflow.tar.gz


<h1> Retreiving preprocessed data </h1>

To save time, we'll go off the preprocessed data from Lab4a. To save time, let's start off from my Lab4a results (which I carried out on the full 1 billion-row dataset).  Change the BUCKET below to be yours.

Tuning is carried out over a segment of the training data (you should not use the validation data for this).

In [11]:
%bash
BUCKET=cloud-training-demos-ml

SOURCE=gs://cloud-training-demos/taxifare/taxi_preproc4a_full
SUFFIX="-of-00003.tfrecord.gz"  
gsutil -m rm -rf gs://$BUCKET/taxifare/taxi_preproc4b/
gsutil cp $SOURCE/metadata.yaml gs://$BUCKET/taxifare/taxi_preproc4b/metadata.yaml
for file in features_train-0000* features_train-0002*; do
    gsutil -m cp $SOURCE/$file gs://$BUCKET/taxifare/taxi_preproc4b/
done

Removing gs://cloud-training-demos-ml/taxifare/taxi_preproc4b/features_train-00000-of-00003.tfrecord.gz#1475059776276255...
Removing gs://cloud-training-demos-ml/taxifare/taxi_preproc4b/features_train-00002-of-00003.tfrecord.gz#1475059782127696...
Removing gs://cloud-training-demos-ml/taxifare/taxi_preproc4b/features_train-00001-of-00003.tfrecord.gz#1475059780244347...
Removing gs://cloud-training-demos-ml/taxifare/taxi_preproc4b/metadata.yaml#1475059773609036...
/ [4/4 objects] 100% Done                                                       
Operation completed over 4 objects.                                              
Copying gs://cloud-training-demos/taxifare/taxi_preproc4a_full/metadata.yaml [Content-Type=text/plain]...
- [1 files][  2.2 KiB/  2.2 KiB]                                                
Operation completed over 1 objects/2.2 KiB.                                      
Copying gs://cloud-training-demos/taxifare/taxi_preproc4a_full/features_train-00000-of-00027.tfrecor

<h2> Modify TensorFlow code </h2>

We want to make the number of buckets and the number of hidden nodes an optimizable parameter.
In order to do this, we have to get them from the command-line.

This shows all the code that now references the number_buckets hyperparameter.

In [25]:
%bash
grep -3 number_buckets taxifare/trainer/*.py

taxifare/trainer/task.py-  parser.add_argument('--metadata_path', type=str)
taxifare/trainer/task.py-  parser.add_argument('--output_path', type=str)
taxifare/trainer/task.py-  parser.add_argument('--max_steps', type=int, default=2000)
taxifare/trainer/task.py:  parser.add_argument('--number_buckets', type=int, default=5)
taxifare/trainer/task.py-  parser.add_argument('--hidden_layer1_size', type=int, default=256)
taxifare/trainer/task.py-
taxifare/trainer/task.py-  args = parser.parse_args()
taxifare/trainer/task.py-  HYPERPARAMS['hidden_layer1_size'] = args.hidden_layer1_size
taxifare/trainer/task.py-  HYPERPARAMS['hidden_layer2_size'] = args.hidden_layer1_size / 2
taxifare/trainer/task.py-  HYPERPARAMS['hidden_layer3_size'] = args.hidden_layer1_size / 4
taxifare/trainer/task.py:  HYPERPARAMS['number_buckets'] = args.number_buckets
taxifare/trainer/task.py-
taxifare/trainer/task.py-  dispatch(args, cluster, task, job)
taxifare/trainer/task.py-
--
taxifare/trainer/task.py-  'hidden_la

We also have to add a summary metric named <b>training/hptuning/metric</b> to the TensorFlow graph.

In [4]:
%bash
grep -3 hptuning taxifare/trainer/task.py

    summary = tf.scalar_summary('accuracy', accuracy_op)

    rmse_op, eval_op2 = metric_ops.streaming_root_mean_squared_error(output, targets)
    summary2 = tf.scalar_summary('training/hptuning/metric', rmse_op)

    saver = tf.train.Saver()



<h2> Train once </h2>

Here, we package up the code and train as normal.

In [14]:
%bash
rm -rf taxifare.tar.gz taxi_trained
tar cvfz taxifare.tar.gz taxifare
gsutil cp taxifare.tar.gz gs://cloud-training-demos-ml/taxifare/source4b/taxifare.tar.gz

taxifare/
taxifare/PKG-INFO
taxifare/setup.cfg
taxifare/setup.py
taxifare/trainer/
taxifare/trainer/__init__.py
taxifare/trainer/task.py
taxifare/trainer/task.py.orig
taxifare/trainer/taxifare.py
taxifare/trainer/taxifare.py.orig
taxifare/trainer.egg-info/
taxifare/trainer.egg-info/dependency_links.txt
taxifare/trainer.egg-info/PKG-INFO
taxifare/trainer.egg-info/SOURCES.txt
taxifare/trainer.egg-info/top_level.txt


Copying file://taxifare.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  8.4 KiB/  8.4 KiB]                                                
Operation completed over 1 objects/8.4 KiB.                                      


In [12]:
%bash
gsutil cp -R gs://cloud-training-demos-ml/taxifare/taxi_preproc4b /content/CPB102/lab4b

Copying gs://cloud-training-demos-ml/taxifare/taxi_preproc4b/features_train-00000-of-00027.tfrecord.gz...
Copying gs://cloud-training-demos-ml/taxifare/taxi_preproc4b/features_train-00001-of-00027.tfrecord.gz...
Copying gs://cloud-training-demos-ml/taxifare/taxi_preproc4b/features_train-00002-of-00027.tfrecord.gz...
Copying gs://cloud-training-demos-ml/taxifare/taxi_preproc4b/features_train-00003-of-00027.tfrecord.gz...
| [4 files][ 24.3 MiB/ 24.3 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://cloud-training-demos-ml/taxifare/taxi_preproc4b/features_train-00004-of-00027.tfrecord.gz...
Copying gs://cloud-training-demos-ml/taxifare/taxi_preproc4b/features_train-00005-of-00027.tfrecord.gz...
Copying gs://cloud-training-dem

In [15]:
%%mlalpha train
package_uris: /content/CPB102/lab4b/taxifare.tar.gz
python_module: trainer.task
scale_tier: BASIC
region: us-central1
args:
  train_data_paths: /content/CPB102/lab4b/taxi_preproc4b/features_train-0000*
  eval_data_paths:  /content/CPB102/lab4b/taxi_preproc4b/features_train-0002*
  metadata_path: /content/CPB102/lab4b/taxi_preproc4b/metadata.yaml
  output_path: /content/CPB102/lab4b/taxi_trained
  max_steps: 200
  hidden_layer1_size: 8
  number_buckets: 2

Job Running...

master log

master: INFO:root:Step 100: loss = 13.58 (0.003 sec) master: INFO:root:Step 200: loss = 16.47 (0.004 sec) master: INFO:root:Final accuracy after 200 steps = 0.921 master: INFO:root:Done training. master: plat = [128, 1] master: latdist = [128, 1] master: pickup = [128, 4] master: pickupdropoff = [128, 16] master: inputs= [128, 34] master: plat = [128, 1] master: latdist = [128, 1] master: pickup = [128, 4] master: pickupdropoff = [128, 16] master: inputs= [128, 34] master: plat = [None, 1] master: latdist = [None, 1] master: pickup = [None, 4] master: pickupdropoff = [None, 16] master: inputs= [None, 34] master:

Job Finished.

In [61]:
%tensorboard start --logdir /content/CPB102/lab4b/taxi_trained

In [62]:
%tensorboard stop --pid 27313

In [9]:
%mlalpha summary --dir /content/CPB102/lab4b/taxi_trained/eval

<h2> Hyperparameter training </h2>

Now, we carry out the training, but this time on the cloud, and this time with some hyperparameters

In [16]:
!gsutil -m -q rm -r gs://cloud-training-demos-ml/taxifare/taxi_trained4b

In [17]:
%%mlalpha train --cloud
package_uris: gs://cloud-training-demos-ml/taxifare/source4b/taxifare.tar.gz
python_module: trainer.task
scale_tier: BASIC
region: us-central1
args:
  train_data_paths: gs://cloud-training-demos-ml/taxifare/taxi_preproc4b/features_train-0001*
  eval_data_paths: gs://cloud-training-demos-ml/taxifare/taxi_preproc4b/features_eval-0002*
  metadata_path: gs://cloud-training-demos-ml/taxifare/taxi_preproc4b/metadata.yaml
  output_path: gs://cloud-training-demos-ml/taxifare/taxi_trained4b
  max_steps: 1000
hyperparameters:
  goal: MINIMIZE
  max_trials: 20
  max_parallel_trials: 3
  params:
    - parameter_name: hidden_layer1_size
      type: INTEGER
      min_value: 32
      max_value: 256
      scale_type: UNIT_LINEAR_SCALE  
    - parameter_name: number_buckets
      type: INTEGER
      min_value: 5
      max_value: 15
      scale_type: UNIT_LINEAR_SCALE  

In [82]:
%mlalpha jobs --name  trainer_task_160927_000547

In [18]:
!gsutil ls gs://cloud-training-demos-ml/taxifare/taxi_trained4b

gs://cloud-training-demos-ml/taxifare/taxi_trained4b/
gs://cloud-training-demos-ml/taxifare/taxi_trained4b/1/
gs://cloud-training-demos-ml/taxifare/taxi_trained4b/10/
gs://cloud-training-demos-ml/taxifare/taxi_trained4b/2/
gs://cloud-training-demos-ml/taxifare/taxi_trained4b/3/
gs://cloud-training-demos-ml/taxifare/taxi_trained4b/4/
gs://cloud-training-demos-ml/taxifare/taxi_trained4b/5/
gs://cloud-training-demos-ml/taxifare/taxi_trained4b/6/
gs://cloud-training-demos-ml/taxifare/taxi_trained4b/7/
gs://cloud-training-demos-ml/taxifare/taxi_trained4b/8/
gs://cloud-training-demos-ml/taxifare/taxi_trained4b/9/


In [8]:
%tensorboard start --logdir gs://cloud-training-demos-ml/taxifare/taxi_trained4b

In [9]:
%tensorboard stop --pid 3222

In [19]:
%mlalpha summary --dir gs://cloud-training-demos-ml/taxifare/taxi_trained4b/*/summaries  gs://cloud-training-demos-ml/taxifare/taxi_trained4b/*/eval  --name loss --step

Copyright 2016 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License